# Importing Overlay and checking available IPs

In [12]:
from pynq import Overlay, allocate
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import time
#from scipy.signal import find_peaks

In [13]:
ol = Overlay("fft_1024.bit")

In [14]:
# ol?

In [15]:
data_channel = ol.axi_dma_0

In [16]:
# data_channel?

# Create Sliders for Amplitude and Frequency

In [17]:
f1 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f1")
f2 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f2")
f3 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f3")
A1 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A1")
A2 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A2")
A3 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A3")
samples = 1024
T = 1

In [18]:
def create_data(A1, A2, A3, f1, f2, f3):
    T = 1
    samples = 1024
    t = np.linspace(0, T, samples)
    data = A1*np.sin(2*np.pi*f1*t) + A2*np.sin(2*np.pi*f2*t) + A3*np.sin(2*np.pi*f3*t)
    return data, t

def plot_graph(A1, A2, A3, f1, f2, f3):
    data, t = create_data(A1, A2, A3, f1, f2, f3)
    plt.plot(t, np.abs(data), label = "Input Data")
    plt.xlabel('Time (s)')
    plt.ylabel('Absolute Value')
    plt.legend()
    plt.show()

In [19]:
widgets.interact_manual(plot_graph, A1=A1, A2=A2, A3=A3, f1=f1, f2=f2, f3=f3)

interactive(children=(FloatSlider(value=100.0, description='A1', max=200.0, min=100.0, step=1.0), FloatSlider(…

<function __main__.plot_graph>

# FFT function with options for PS PL Implementation

In [20]:
def plot_fft(A1, A2, A3, f1, f2, f3, ps, pl):
    data, t = create_data(A1, A2, A3, f1, f2, f3)
    freq = np.fft.fftshift(np.fft.fftfreq(samples,T/samples))
    
    if ps:
        st = time.time()
        ps_output = np.abs(np.fft.fft(data, samples)) / samples
        et = time.time()
        
        print("PS Execution Time: " + str(et - st))
        
        ps_output_shifted = np.fft.fftshift(ps_output)
        plt.plot(freq, ps_output_shifted, label='PS Output')
        
    if pl:
        input_buffer = allocate(1024, np.csingle)
        output_buffer = allocate(1024,np.csingle)
        np.copyto(input_buffer, data)
        
        st = time.time()
        data_channel.sendchannel.transfer(input_buffer)
        data_channel.recvchannel.transfer(output_buffer)
        data_channel.sendchannel.wait()
        data_channel.recvchannel.wait()
        et = time.time()
        
        print("PL Execution Time: " + str(et - st))
        
        pl_output_shifted = np.fft.fftshift(np.abs(output_buffer) / samples)
        plt.plot(freq, pl_output_shifted, label='PL Output')
        
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.legend()
    plt.show()

In [21]:
c1 = widgets.Checkbox(value=True, description="PS Implementation")
c2 = widgets.Checkbox(value=True, description="PL Implementation")

In [22]:
widgets.interact_manual(plot_fft, A1=A1, A2=A2, A3=A3, f1=f1, f2=f2, f3=f3, ps=c1, pl=c2)

interactive(children=(FloatSlider(value=100.0, description='A1', max=200.0, min=100.0, step=1.0), FloatSlider(…

<function __main__.plot_fft>